# Introduction & problem description

## Introduction

### Millions of people are using Twitter and expressing their emotions like happiness, sadness, angry, etc. The Sentiment analysis is also about detecting the emotions, opinion, assessment, attitudes, and took this into consideration as a way humans think. Sentiment analysis classifies the emotions into classes such as positive or negative. Nowadays, industries are interested to use textual data for semantic analysis to extract the view of people about their products and services. Sentiment analysis is very important for them to know the customer satisfaction level and they can improve their services accordingly. To work on the text data, they try to extract the data from social media platforms. There are a lot of social media sites like Google Plus, Facebook, and Twitter that allow expressing opinions, views, and emotions about certain topics and events. Microblogging site Twitter is expanding rapidly among all other online social media networking sites with about 200 million users. Twitter was founded in 2006 and currently, it is the most famous microblogging platform. In 2017 2 million users shared 8.3 million tweets in one hour. Twitter users use to post their thoughts, emotions, and messages on their profiles, called tweets. Words limit of a single tweet has 140 characters. Twitter sentiment analysis based on the NLP (natural language processing) field. For tweets text, we use NLP techniques like tokenizing the words, removing the stop words like I, me, my, our, your, is, was, etc. Natural language processing also plays a part to preprocess the data like cleaning the text and removing the special characters and punctuation marks. Sentimental analysis is very important because we can know the trends of people’s emotions on specific topics with their tweets.



## Problem description

### To devise a sentimental analyzer for overcoming the challenges to identify the twitter tweets text sentiments (positive, negative) by using naive bayes , support vector classifier and logistic regression

## Labraries importing

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from nltk.tokenize import RegexpTokenizer
import re
import string
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings

warnings.filterwarnings("ignore")

## Loading data

In [ ]:
# loading data set
data = pd.read_csv("data set.csv", encoding="ISO-8859-1", engine="python")
data.columns = ["Sentiment", "Time", "Date", "Query", "Username", "Tweet"]

# Exploratory data analysis 

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
# length of data
print("lenght of data is :", len(data))

In [ ]:
# Shape of data
data.shape

In [ ]:
# Information about data
data.info()

In [ ]:
# Data types of all features
data.dtypes

In [ ]:
# Checking Null values
data.isnull().mean()

In [ ]:
# Describtion for data
data.describe()

Mohammed Saeed A5oya